# Homework 2

# Set up

## Installing packages

In [1]:
!pip install requests PyPDF2 gdown
!pip install 'markitdown[pdf]'
!pip install langchain_mcp_adapters langchain_google_genai langchain-openai

## Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `VERTEX_API_KEY`.


1.   Look for the key icon on the left panel of your colab.
2.   Under `Name`, create `VERTEX_API_KEY`.
3. Copy your key to `Value`.

If you cannot use VERTEX_API_KEY, you can use deepseek models via `DEEPSEEK_API_KEY`. It does not affect your score.



In [2]:
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('GEMINI_API_KEY')
# DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')

# Download sample CVs

## Downloading sample_cv.pdf
The codes below download the sample CV


In [3]:
import os
import gdown

folder_id = "1adYKq7gSSczFP3iikfA8Er-HSZP6VM7D"
folder_url = f"https://drive.google.com/drive/folders/{folder_id}"

output_dir = "downloaded_cvs"
os.makedirs(output_dir, exist_ok=True)

gdown.download_folder(
    url=folder_url,
    output=output_dir,
    quiet=False,
    use_cookies=False
)

Retrieving folder contents


Processing file 1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp CV_1.pdf
Processing file 16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs CV_2.pdf
Processing file 15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr CV_3.pdf
Processing file 1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk CV_4.pdf
Processing file 1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C CV_5.pdf


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp
To: /content/downloaded_cvs/CV_1.pdf
100%|██████████| 147k/147k [00:00<00:00, 25.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs
To: /content/downloaded_cvs/CV_2.pdf
100%|██████████| 75.1k/75.1k [00:00<00:00, 54.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr
To: /content/downloaded_cvs/CV_3.pdf
100%|██████████| 72.0k/72.0k [00:00<00:00, 29.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk
To: /content/downloaded_cvs/CV_4.pdf
100%|██████████| 73.3k/73.3k [00:00<00:00, 21.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C
To: /content/downloaded_cvs/CV_5.pdf
100%|██████████| 97.9k/97.9k [00:00<00:00, 39.7MB/s]
Download complete

['downloaded_cvs/CV_1.pdf',
 'downloaded_cvs/CV_2.pdf',
 'downloaded_cvs/CV_3.pdf',
 'downloaded_cvs/CV_4.pdf',
 'downloaded_cvs/CV_5.pdf']

In [4]:
# =====================================================
#  Load and display all CV PDFs in order
# =====================================================
import os
from markitdown import MarkItDown

cv_dir = "downloaded_cvs"

# Initialize MarkItDown
md = MarkItDown(enable_plugins=False)

# Collect and sort PDFs numerically
pdf_files = sorted(
    [f for f in os.listdir(cv_dir) if f.lower().endswith(".pdf")],
    key=lambda x: int("".join(filter(str.isdigit, x)))  # CV_1.pdf → 1
)

all_cvs = []

for pdf_name in pdf_files:
    pdf_path = os.path.join(cv_dir, pdf_name)
    result = md.convert(pdf_path)

    all_cvs.append({
        "file": pdf_name,
        "text": result.text_content
    })

    print("=" * 80)
    print(f"📄 {pdf_name}")
    print("=" * 80)
    print(result.text_content)
    print("\n\n")


📄 CV_1.pdf
|     |     |     |     | John         |           | Smith        |                   |     |     |
| --- | --- | --- | --- | ------------ | --------- | ------------ | ----------------- | --- | --- |
|     |     |     |     | Marketing    |           | Professional |                   |     |     |
|     |     |     |     | + Singapore, | Singapore |              | (cid:209) Kowloon |     |     |
Experience
|                |                  |     |          |                     |              |            |     | 2020 – | Present |
| -------------- | ---------------- | --- | -------- | ------------------- | ------------ | ---------- | --- | ------ | ------- |
| Engineer,      | ByteDance        |     |          |                     |              |            |     |        |         |
| • Worked       | in a fast-paced, |     | global   | technology          | environment. |            |     |        |         |
| • Collaborated | across           |     | teams to | sup

# Connect to our MCP server

Documentation about MCP: https://modelcontextprotocol.io/docs/getting-started/intro.

Using MCP servers in Langchain https://docs.langchain.com/oss/python/langchain/mcp.

## Check which tools that the MCP server provide

In [5]:
import asyncio
import json
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
for tool in mcp_tools:
    print(tool.name)
    print(tool.description)
    print(tool.args)
    print("\n\n------------------------------------------------------\n\n")

search_facebook_users
Search for Facebook users by display name (supports partial and fuzzy matching).

Args:
    q: Search query string (case-insensitive, matches any part of display name)
       Examples: "John", "john smith", "Smith"
    limit: Maximum number of results to return (default: 20, max: 20)
    fuzzy: Enable fuzzy matching if exact search returns no results (default: True)

Returns:
    List of user dictionaries, each containing:
    - id (int): Unique Facebook user ID for use with get_facebook_profile()
    - display_name (str): User's Facebook display name (may differ from legal name)
    - city (str): Current city of residence
    - country (str): Country of residence
    - match_type (str): "exact" or "fuzzy" (indicates search method used)
    
    Returns empty list [] if no matches found.

Example:
    search_facebook_users("Alex Chan", limit=5)
    → [{"id": 123, "display_name": "Alex Chan", "city": "Hong Kong", "country": "Hong Kong", "match_type": "exact"}]
    

## A simple agent using tools from the MCP server


In [6]:
import os
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient

# ---------------------------
# 1. Define a local tool
# ---------------------------
@tool
def say_hello(name: str) -> str:
    """Say hello to a person by name."""
    return f"Hello, {name}! 👋"

# ---------------------------
# 2. Load MCP tools + merge
# ---------------------------
client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
tools = mcp_tools + [say_hello]

# ---------------------------
# 3. Initialize Gemini (tool-enabled) or deepseek
# ---------------------------
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=GEMINI_VERTEX_API_KEY,
    temperature=0,
)

# from langchain_openai import ChatOpenAI
# DEEPSEEK_API_KEY = userdata.get("DEEPSEEK_API_KEY")
# llm = ChatOpenAI(
#     model="deepseek-chat",          # or "deepseek-reasoner"
#     api_key=DEEPSEEK_API_KEY,
#     base_url="https://api.deepseek.com/v1",
#     temperature=0,
# )

llm_with_tools = llm.bind_tools(tools)

# ---------------------------
# 4. Single-step invocation
# ---------------------------
query = "Say hello to Bao using tool, then search for someone named Alice on Facebook."

response = llm_with_tools.invoke([
    HumanMessage(content=query)
])

print(response)

content='' additional_kwargs={'function_call': {'name': 'search_facebook_users', 'arguments': '{"q": "Alice"}'}} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019c905f-9e80-7970-886c-63ea771ff803-0' tool_calls=[{'name': 'say_hello', 'args': {'name': 'Bao'}, 'id': '05e7f866-33e4-49d0-a337-2337cef3d731', 'type': 'tool_call'}, {'name': 'search_facebook_users', 'args': {'q': 'Alice'}, 'id': '25f5b7f1-9100-4055-b56c-d9b4e9d2c599', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 2852, 'output_tokens': 33, 'total_tokens': 2885, 'input_token_details': {'cache_read': 0}}


In [7]:
# This block provides you some tests to get faminilar with our MCP server

# # Test 1: Search Facebook users (exact match)
# await tools[0].ainvoke({'q': "Alex Chan", 'limit': 5})

# # Test 2: Search Facebook users (fuzzy match with typo)
# await tools[0].ainvoke({'q': "Alx Chn", 'limit': 5, 'fuzzy': True})

# # Test 3: Get Facebook profile
# await tools[1].ainvoke({'user_id': 123})

# # Test 4: Get Facebook mutual friends
# await tools[2].ainvoke({'user_id_1': 123, 'user_id_2': 456})

# # Test 5: Search LinkedIn people (exact match)
# await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5})

# # Test 6: Search LinkedIn people (fuzzy match with typo)
# await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5, 'fuzzy': True})

# # Test 7: Get LinkedIn profile
# await tools[4].ainvoke({'person_id': 456})

# Test 8: Get LinkedIn interactions
await tools[5].ainvoke({'person_id': 456})

[{'type': 'text',
  'text': '{"profile_id":456,"post_count":4,"total_likes":5,"liked_by":[4390,3622,7500,4269,8464],"engagement_score":1.25}',
  'id': 'lc_2e2cb789-0147-4a0c-b02e-4f361cae4d08'}]

# Evaluation code

In the test phase, you will be given 5 CV files with fixed names:

    CV_1.pdf, CV_2.pdf, CV_3.pdf, CV_4.pdf, CV_5.pdf

Your system must process these CVs and output a list of 5 scores,
one score per CV, in the same order:

    scores = [s1, s2, s3, s4, s5]

Each score must be a float in the range [0, 1], representing the
reliability or confidence that the CV is valid (or meets the task criteria).

The ground-truth labels are binary:

    groundtruth = [0 or 1, ..., 0 or 1]

Each CV is evaluated independently using a threshold of 0.5:

- If score > 0.5 and groundtruth == 1 → Full credit
- If score ≤ 0.5 and groundtruth == 0 → Full credit
- Otherwise → No credit

In other words, 0.5 is the decision threshold.

- Each CV contributes equally.
- Final score = (number of correct decisions) / 5


In [8]:
# =====================================================
#  Evaluation code
# =====================================================

def evaluate(scores, groundtruth, threshold=0.5):
    """
    scores: list of floats in [0, 1], length = 5
    groundtruth: list of ints (0 or 1), length = 5
    """
    assert len(scores) == 5
    assert len(groundtruth) == 5

    correct = 0
    decisions = []

    for s, gt in zip(scores, groundtruth):
        pred = 1 if s > threshold else 0
        decisions.append(pred)
        if pred == gt:
            correct += 1

    final_score = correct / len(scores)

    return {
        "decisions": decisions,
        "correct": correct,
        "total": len(scores),
        "final_score": final_score
    }


In [12]:
import json
import asyncio # Make sure asyncio is imported for the sleep function
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage

async def verify_cv_async(cv_text, filename, mcp_tools, llm_with_tools):
    """
    Runs an asynchronous multi-turn agent loop to verify a CV and generate a comprehensive report.
    Includes network retry logic to bypass 503 Server Errors.
    """
    system_prompt = """
    You are an advanced KYC & Background Check Agent. Your task is to perform Deep Verification of a candidate's CV using a full suite of LinkedIn and Facebook tools, and generate a comprehensive verification report.

    COMPREHENSIVE VERIFICATION PROTOCOL:

    PHASE 1: PROFESSIONAL DISCOVERY (LINKEDIN)
    - Search LinkedIn using ONLY the candidate's exact name. (For generic names, search by Company Name instead).
    - Fetch the profiles. ITERATION RULE: If a profile shares NO specific companies or universities with the CV, it is a NAME COLLISION. Discard it and fetch the NEXT ID.

    PHASE 2: PROFESSIONAL AUTHENTICITY (LINKEDIN)
    - Use `get_linkedin_interactions` to verify their network strength and professional engagement.

    PHASE 3: CROSS-PLATFORM DISCOVERY (FACEBOOK)
    - Attempt to corroborate the candidate's identity on Facebook using `search_facebook_users` and `get_facebook_profile`.

    PHASE 4: SOCIAL PROOFING (FACEBOOK)
    - If found on Facebook, use `get_facebook_mutual_friends` to assess social authenticity.

    PHASE 5: SCORING (Output a float from 0.0 to 1.0)
       - 0.8 to 1.0 (Pass): Core identity matches across platforms. Minor typos or 1-year date shifts are fine.
       - 0.5 to 0.7 (Pass with penalty): Core identity matches, but there are injected inconsistencies (e.g., CV says "Present" but profile says "is_current: false", or profile status is "student"). YOU MUST SCORE EXACTLY 0.6 FOR THESE.
       - 0.0 to 0.4 (Fail): Major fabrications (e.g., A claimed job is COMPLETELY MISSING, or claiming a PhD but only having an MSc). Or the profile is completely invisible.

    PHASE 6: REPORT GENERATION
    End your turn with this exact JSON format ONLY:
    {
        "score": 0.6,
        "report": {
            "candidate_name": "Extracted Name",
            "identity_confirmation": "How did you confirm this is the right person? (e.g., Matched BCG and HKU)",
            "linkedin_findings": "Detailed analysis of LinkedIn work/education matches and interactions.",
            "facebook_findings": "Detailed analysis of Facebook cross-platform consistency and mutual friends.",
            "discrepancies_found": [
                "List item 1",
                "List item 2"
            ],
            "final_conclusion": "Overall summary of the verification."
        }
    }
    """

    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Please perform deep verification and generate a report for this CV:\n\n{cv_text}")
    ]

    print(f"==================================================")
    print(f"🔍 INITIATING DEEP VERIFICATION: {filename}")
    print(f"==================================================")

    for step in range(20):
        response = await llm_with_tools.ainvoke(messages)
        messages.append(response)

        if response.tool_calls:
            print(f"  [Step {step+1}] 🛠️ Tools Deployed: {', '.join([tc['name'] for tc in response.tool_calls])}")
            for tool_call in response.tool_calls:
                selected_tool = next((t for t in mcp_tools if t.name == tool_call["name"]), None)
                if selected_tool:

                    # --- ADDED: 3-Attempt Network Retry Wrapper ---
                    tool_result = None
                    for attempt in range(3):
                        try:
                            tool_result = await selected_tool.ainvoke(tool_call["args"])
                            break # Success! Break out of the retry loop.
                        except Exception as e:
                            if attempt < 2:
                                print(f"    ⚠️ Network Error on {tool_call['name']}. Retrying in 2 seconds...")
                                await asyncio.sleep(2) # Wait for ngrok to recover
                            else:
                                tool_result = f"Error: {str(e)}" # Only fail after 3 tries
                    # ----------------------------------------------

                    messages.append(ToolMessage(
                        content=str(tool_result),
                        tool_call_id=tool_call["id"],
                        name=tool_call["name"]
                    ))
        else:
            raw_content = response.content

            text_content = ""
            if isinstance(raw_content, list):
                for block in raw_content:
                    if isinstance(block, dict) and block.get("type") == "text":
                        text_content += block.get("text", "") + " "
                    elif isinstance(block, str):
                        text_content += block + " "
            else:
                text_content = str(raw_content)

            try:
                cleaned_output = text_content.replace("```json", "").replace("```", "").strip()
                result_json = json.loads(cleaned_output)

                score = float(result_json.get("score", 0.0))
                report = result_json.get("report", {})

                print("\n✅ VERIFICATION COMPLETE")
                print(f"📊 CONFIDENCE SCORE: {score}/1.0")
                print(f"👤 CANDIDATE: {report.get('candidate_name', 'Unknown')}")
                print(f"🔑 IDENTITY MATCH: {report.get('identity_confirmation', 'None')}")
                print(f"\n🔵 LINKEDIN ANALYSIS:\n{report.get('linkedin_findings', 'N/A')}")
                print(f"\n🔵 FACEBOOK ANALYSIS:\n{report.get('facebook_findings', 'N/A')}")

                discrepancies = report.get('discrepancies_found', [])
                print("\n⚠️ DISCREPANCIES FOUND:")
                if not discrepancies:
                    print("  - None found.")
                else:
                    for d in discrepancies:
                        print(f"  - {d}")

                print(f"\n📝 CONCLUSION:\n{report.get('final_conclusion', 'N/A')}")
                print(f"==================================================\n")

                return score

            except Exception as e:
                messages.append(HumanMessage(content="Please provide your final answer in the requested JSON report format ONLY."))
                print(f"  [Step {step+1}] ⚠️ Formatting Report...")

    print(f"❌ {filename} Failed: Exceeded 20 steps.")
    return 0.0

In [13]:
scores = [] # Your code should generate this list [0.2, 0.3, 0.4, 0.5, 0.6]
for cv in all_cvs:
    score = await verify_cv_async(cv["text"], cv["file"], mcp_tools, llm_with_tools)
    scores.append(score)

groundtruth = [1, 1, 1, 0, 0] # Do not modify

result = evaluate(scores, groundtruth)
print(result)


🔍 INITIATING DEEP VERIFICATION: CV_1.pdf
  [Step 1] 🛠️ Tools Deployed: search_linkedin_people
  [Step 2] 🛠️ Tools Deployed: get_linkedin_profile
  [Step 3] 🛠️ Tools Deployed: get_linkedin_interactions
  [Step 4] 🛠️ Tools Deployed: search_facebook_users
  [Step 5] 🛠️ Tools Deployed: get_facebook_profile

✅ VERIFICATION COMPLETE
📊 CONFIDENCE SCORE: 0.6/1.0
👤 CANDIDATE: John Smith
🔑 IDENTITY MATCH: Candidate's name 'John Smith' and location 'Singapore, Singapore' matched across CV, LinkedIn, and Facebook. Education at 'McGill University' also matched on LinkedIn.

🔵 LINKEDIN ANALYSIS:
LinkedIn profile (ID 9) matches the candidate's name, headline ('Marketing Professional'), industry ('Marketing'), and location ('Singapore, Singapore'). The ByteDance experience (Engineer, 2020) and McGill University education (BSc in Marketing, graduated 2009) also align with the CV. However, the LinkedIn profile lists the ByteDance role as 'is_current: false' despite the CV stating 'Present'. Additionally